In [5]:
import math
import numpy as np
import time
import random

from multiprocessing import Pool

%load_ext autoreload
%autoreload 2
from tools import *

start_time = time.perf_counter_ns()

def process_time():
    print(f'{(time.perf_counter_ns() - start_time) / 1000} ms')

def fib_math(n):
    return int((1 / math.sqrt(5)) * (((1 + math.sqrt(5))/2) ** n - ((1 - math.sqrt(5))/2) ** n))

def fib_loop(n):
    _cur, _next = 0, 1
    for _ in range(n):
        _cur, _next = _next, _cur + _next
    return _cur

def fib_recursive(n):
    if n <= 1: return n
    return fib_recursive(n - 1) + fib_recursive(n - 2)

from functools import lru_cache

@lru_cache
def fib_memoization(n):
    if n <=1: return n
    return fib_memoization(n - 1) + fib_memoization(n - 2)

n = 94


print(fib_math(n))
print(fib_loop(n))
# process_time()

print(fib_memoization(n))
# process_time()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
19740274219868282880
19740274219868223167
19740274219868223167


In [41]:
print(fib_math(80))
print(fib_math(81))
print(fib_math(82))
print(fib_math(83))


23416728348467744
37889062373144008
61305790721611760
99194853094755776


In [3]:
n = 30
for _ in range(16):
    print(fib_recursive(n), end=' ')

832040 832040 832040 832040 832040 832040 832040 832040 832040 832040 832040 832040 832040 832040 832040 832040 

In [6]:
n = 35
with Pool(16) as p:
    print(p.map(fib_recursive, (n, n, n, n, n, n, n, n, n, n, n, n, n, n, n, n)))

[9227465, 9227465, 9227465, 9227465, 9227465, 9227465, 9227465, 9227465, 9227465, 9227465, 9227465, 9227465, 9227465, 9227465, 9227465, 9227465]


In [9]:
dt = {'right':(1, 0), 'left':(-1, 0), 'up':(0, -1), 'down':(0, 1)}
print(list(dt.keys()))


['right', 'left', 'up', 'down']


- 멀티프로세싱을 통하면 풀 개수만큼 나눠 가능하다.
    - 프로세싱을 멀티플라이 하는 코스트 자체가 제법 되어서 단일 작업보다 좀 더 걸린다.

In [15]:
import numpy as np
import random
DIRECTIONS = {'right':(1, 0), 'left':(-1, 0), 'up':(0, -1), 'down':(0, 1)}

class GridWorld:
    def __init__(self, r=4, c=4):
        self.row = 0
        self.col = 0
        self.stage = np.zeros((r, c))
        self.row_size = r
        self.col_size = c
    
    def step(self, direction:str):
        '''direction 0/right, 1/left, 2/up, 3/down'''
        if self.move(direction):
            reward = -1
            done = self.is_done()
        return (self.row, self.col), reward, done
    
    def is_done(self):
        # print(self.row, self.col)
        return (self.row == self.stage.shape[0] and self.col == self.stage.shape[1])
    
    def move(self, direction):
        if direction not in DIRECTIONS.keys(): return False
        self.row = clamp(self.row + DIRECTIONS[direction][0], 0, self.stage.shape[0])
        self.col = clamp(self.col + DIRECTIONS[direction][1], 0, self.stage.shape[1])
        return True
    
    def get_state(self):
        return self.row, self.col
    
    def reset(self):
        # self.__init__(*self.stage.shape)
        self.row = 0
        self.col = 0
        return (self.row, self.col)
        
class Agent:
    def __init__(self):
        pass
    
    def select_action(self):
        return random.choice(list(DIRECTIONS.keys()))

def main():
    env = GridWorld()
    agent = Agent()
    gamma = 1.0
    reward = -1
    alpha = 0.001
    
    for k in range(2):
        done = False
        history = []
        
        while not done:
            action = agent.select_action()
            (x, y), reward, done = env.step(action)
            history.append((x, y, reward))
        print(history)
        env.reset()
        
        cum_reward = 0
        for transition in history[::-1]:
            x, y, reward = transition
            env.stage[x][y] += alpha * (cum_reward - env.stage[x][y])
            cum_reward = reward + gamma * cum_reward
            
        # for row in env.stage:
            # print(row)
            
    

# ttt = GridWorld()
# ttt.move('right')
# ttt.reset()
# print(ttt.row, ttt.col)

main()

[(0, 0, -1), (0, 1, -1), (0, 1, -1), (0, 1, -1), (0, 1, -1), (1, 1, -1), (1, 0, -1), (1, 0, -1), (2, 0, -1), (2, 0, -1), (3, 0, -1), (3, 1, -1), (3, 0, -1), (2, 0, -1), (2, 1, -1), (1, 1, -1), (1, 0, -1), (0, 0, -1), (0, 1, -1), (1, 1, -1), (2, 1, -1), (2, 0, -1), (2, 1, -1), (2, 0, -1), (1, 0, -1), (1, 1, -1), (0, 1, -1), (0, 1, -1), (0, 2, -1), (0, 3, -1), (1, 3, -1), (1, 4, -1), (1, 3, -1), (1, 4, -1), (2, 4, -1), (2, 4, -1), (2, 3, -1), (2, 2, -1), (1, 2, -1), (0, 2, -1), (0, 3, -1), (1, 3, -1), (2, 3, -1), (2, 4, -1), (3, 4, -1), (4, 4, -1)]


IndexError: index 4 is out of bounds for axis 0 with size 4